Датасет IMDB Dataset Split из kaggle - https://www.kaggle.com/datasets/ducanger/imdb-dataset?resource=download&select=test.csv

Я невнимательно прочитала задание и заметила это, когда было поздно... вместо того, чтобы смотреть, каких слов в отзыве больше - из положительного списка или из отрицательного, сделала логистической регрессией. Но в конце все равно есть деление на эти списки, я просто не успела доделать это тем способом, которым надо было

In [1]:
from nltk.tokenize import wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
import pandas as pd

In [3]:
morph = MorphAnalyzer()

In [4]:
from collections import Counter

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

In [6]:
def lemmatize(inputfile):
    inputtext = []
    for text in inputfile:
        word_list = nltk.word_tokenize(text)
        lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
        inputtext.append(lemmatized_output)
    return inputtext

In [7]:
train = pd.read_csv("train.csv") 
test = pd.read_csv("test.csv") # прочитала файлы
lemma_test = lemmatize(test['review'])
lemma_train = lemmatize(train['review']) #лемматизировала файлы

In [8]:
cvec = CountVectorizer(binary=True, max_features=10000) 
cvec.fit(lemma_train)

X_train = cvec.transform(lemma_train).astype(int)
X_test = cvec.transform(lemma_test).astype(int)

In [9]:
feature_names = cvec.get_feature_names()

In [10]:
# использую логиситическую регрессию
from sklearn.linear_model import LogisticRegression

y_train = [bool(i) for i in train["sentiment"]] # завожу переменную для ответов, где True = полож,, False = отриц. отзыв


In [11]:
model = LogisticRegression()

model.fit(X_train, y_train) # завожу модель, обучаю


/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [12]:
from sklearn.metrics import accuracy_score, f1_score # загружаю метрики

In [13]:
y_pred = model.predict(X_test) # предсказываю ответ
y_test = [bool(i) for i in test["sentiment"]]

In [14]:
pred_list = y_pred.tolist() # делаю из array список
true_pred_list = [] 
for prediction in pred_list:
    if prediction == True:
        true_pred_list.append(1)
    if prediction == False:
        true_pred_list.append(0) # меняю True и False на 1 и 0 соответственно
result = test[["review"]].copy()
result["sentiment"] = true_pred_list 
result.to_csv("resultnew.csv", index=False) # создаю csv табличку с результатами


In [15]:
from sklearn.metrics import accuracy_score, f1_score
accuracy_score(y_test, y_pred)

0.8711

In [17]:
import csv 

df = pd.read_csv("train.csv", sep=",")
df =df.head(100000)

dictrev = pd.Series(df.sentiment.values,index=df.review).to_dict() #сделала из csv dict
posrev = []
negrev = []
for key, value in dictrev.items(): #добавляю все положительные отзывы в один список, все отрицательные - в другой
    if value == 1:
        posrev.append(key)
    if value == 0:
        negrev.append(key)
posrev = " ".join(posrev)
negrev = " ".join(negrev)


In [ ]:
tokenized_posrev = nltk.word_tokenize(posrev)
tokenized_negrev = nltk.word_tokenize(negrev) #токенизирую  отзывы
onlypos = []
onlyneg = []
for word in tokenized_posrev: # делаю списки, где только те слова, которых нет в другом списке
    if word not in tokenized_negrev:
        onlypos.append(word)
for word in tokenized_negrev:
    if word not in tokenized_posrev:
        onlyneg.append(word)


In [ ]:
posunigrams = Counter(onlypos)
negunigrams = Counter(onlyneg)
print(posunigrams)
print(negunigrams)